In [2]:
from scenedetect import VideoManager, SceneManager
from scenedetect.detectors import ContentDetector

def detect_scene_changes(video_path):
    # Initialize video manager and scene manager
    video_manager = VideoManager([video_path])
    scene_manager = SceneManager()

    # Add ContentDetector (content-based scene detection algorithm)
    scene_manager.add_detector(ContentDetector())

    # Start the video manager
    video_manager.start()

    # Perform scene detection
    scene_manager.detect_scenes(frame_source=video_manager)

    # Get list of scene boundaries
    scene_list = scene_manager.get_scene_list()

    # Extract frame numbers
    frame_numbers = [scene[0].get_frames() for scene in scene_list]

    # Release resources
    video_manager.release()

    # Return list of frame numbers
    return frame_numbers

c:\Users\Marko\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import mido
from mido import MetaMessage, MidiFile, MidiTrack

def create_midi_with_markers(frame_numbers, output_filename, fps=25, bpm=60):
    # Video properties
    ticks_per_beat = 960  # Pro Tools uses 960 ticks per beat
    ticks_per_frame = ticks_per_beat / fps  # Ticks per frame

    # Create a new MIDI file and a single track
    mid = MidiFile(ticks_per_beat=ticks_per_beat)
    track = MidiTrack()
    mid.tracks.append(track)

    # Convert BPM to tempo
    tempo = mido.bpm2tempo(bpm)

    # Add tempo and time signature to the track
    track.append(MetaMessage('set_tempo', tempo=tempo))
    track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

    # Add markers at each scene change frame number
    previous_tick = 0
    for i, frame in enumerate(frame_numbers):
        # Calculate the tick difference from the previous frame
        if i == 0:
            tick_diff = 0
        else:
            tick_diff = int((frame - frame_numbers[i - 1]) * ticks_per_frame)
        
        marker_text = f'SC {i + 1}'  # Custom marker text for each scene
        
        # Add a marker (meta event) at the scene change time
        track.append(MetaMessage('marker', text=marker_text, time=tick_diff))
        previous_tick += tick_diff

    # Save the MIDI file with the provided name
    mid.save(output_filename)
    print(f"MIDI file '{output_filename}' created successfully.")

In [4]:
# Path to the input video file
video_path = 'bele_rade_1080.mp4'

print(f"Detecting scene changes in '{video_path}'...")
# Call the function to detect scene changes
frame_numbers = detect_scene_changes(video_path)

print(f"Finished detecting {len(frame_numbers)} scenes in '{video_path}'.")

print(frame_numbers)



# print(f"Creating MIDI file with markers for '{video_path}'...")

# output_filename = 'magla_v2.mid'
# create_midi_with_markers(frame_numbers, output_filename)

VideoManager is deprecated and will be removed.


Detecting scene changes in 'bele_rade_1080.mp4'...
Finished detecting 5 scenes in 'bele_rade_1080.mp4'.
[0, 300, 533, 1121, 1778]


In [5]:
# Output the frame_numbers array into a file
output_file = 'frame_numbers2.txt'
with open(output_file, 'w') as f:
    for frame in frame_numbers:
        f.write(f"{frame}\n")

In [7]:
# Read the numbers from 'last_frames.txt' and load them into an array
input_file = 'frame_numbers2.txt'
loaded_frame_numbers = []

with open(input_file, 'r') as f:
    for line in f:
        loaded_frame_numbers.append(int(line.strip()))

print(loaded_frame_numbers)

[0, 300, 533, 1121, 1778]


In [8]:
import mido
from mido import MetaMessage, MidiFile, MidiTrack


tpr = 960 / 25

adjusted_frame_numbers = [round(frame * tpr) for frame in loaded_frame_numbers]

# Create a new MIDI file and a single track
mid = MidiFile(ticks_per_beat=960)
track = MidiTrack()
mid.tracks.append(track)

# Convert BPM to tempo
tempo = mido.bpm2tempo(60)

# Add tempo and time signature to the track
track.append(MetaMessage('set_tempo', tempo=tempo))
track.append(MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0))

# Add markers at each scene change frame number
previous_tick = 0
for i, frame in enumerate(adjusted_frame_numbers):
    # Calculate the tick difference from the previous frame
    if i == 0:
        tick_diff = 0
    else:
        tick_diff = frame - adjusted_frame_numbers[i - 1]
    
    marker_text = f'SC {i + 1}'  # Custom marker text for each scene
    
    # Add a marker (meta event) at the scene change time
    track.append(MetaMessage('marker', text=marker_text, time=tick_diff))
    previous_tick += tick_diff

# Save the MIDI file with the provided name
mid.save("test_output23.mid")
print(f"MIDI file magla_round_precise.mid created successfully.")

MIDI file magla_round_precise.mid created successfully.


In [5]:
import cv2

def get_video_fps(video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    
    # Get the FPS (frames per second) of the video
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    
    # Release the video capture object
    video_capture.release()
    
    return fps

# Example usage
video_path = 'test_files/magla.mp4'
fps = get_video_fps(video_path)
print(f"FPS of the video: {fps}")

FPS of the video: 25.0
